In [1]:
import numpy as np
import pandas as pd

print("numpy  version: ", np.__version__)
print("pandas version: ", pd.__version__)

numpy  version:  1.23.5
pandas version:  2.1.4


# Часть 2. Индексация

По уже вполне сложившейся традиции начнём с углубления знаний об индексах, начиная с более простого типа объектов - с Series:

In [21]:
s = pd.Series(
    [0.1, 0.3, 0.5, 1.0], 
    index=["a", "b", "c", "d"]
)

s

a    0.1
b    0.3
c    0.5
d    1.0
dtype: float64

Ранее было сказано, что Series можно рассматривать в качестве обобщения словарей Python'а, что приводти к тому, что мы можем выполнять с индексом всё те же операции, что были присущи объектам типа dict:

In [22]:
# проверка, есть ли ключ
"a" in s, "z" in s

(True, False)

In [23]:
# доступ ко всем ключам
s.keys()

Index(['a', 'b', 'c', 'd'], dtype='object')

In [24]:
# метод items()
list(s.items())

[('a', 0.1), ('b', 0.3), ('c', 0.5), ('d', 1.0)]

In [25]:
# добавление нового элемента
s["e"] = 3.14
s

a    0.10
b    0.30
c    0.50
d    1.00
e    3.14
dtype: float64

По аналогии с массивами можно использовать следующие возможности для индексации и slicing'а:

In [26]:
# slicing по явно заданному индексу
# (обратите внимание, что теперь конец включается)
s["a":"c"]

a    0.1
b    0.3
c    0.5
dtype: float64

In [27]:
# slicing по явно неявному индексу
s[0:2]

a    0.1
b    0.3
dtype: float64

In [28]:
# использование булевых маск
s[(s > 0.2) & (s < 1.0)]

b    0.3
c    0.5
dtype: float64

In [29]:
# в качестве индекса можно подавать список ключей
s[["a", "e"]]

a    0.10
e    3.14
dtype: float64

К текущему моменту может сложиться впечатление (и это вполне обосновано), что индексы - достаточно запутанная история. Чтобы до конца устранить возможное непонимание, давайте рассмотрим ещё один пример:

In [30]:
df = pd.Series(
    ["a", "b", "c"], 
    index=[1, 3, 5]
)
df

1    a
3    b
5    c
dtype: object

Обратите внимание, что индекс в текущем примере состоит из нечётных чисел 1, 3 и 5 (explicit index - явный индекс):

In [31]:
# при индексации используется явный индекс
df[1]

'a'

In [32]:
# slicing использует неявный индекс, 
# поэтому выдаёт элементы со второго по четвёртый (не включая его)
df[1:3]

3    b
5    c
dtype: object

Чтобы устранить эту путаницу, в Pandas есть специальные индексеры. Они используются, чтобы явно выбрать тот индекс, который будет использоваться. Это позволяет не держать в голове нюансы того, explicit или implicit index будет применён

Первый из них - `loc`, которые применяется для explicit index:

In [33]:
df.loc[1]

'a'

In [34]:
df.loc[1:3]

1    a
3    b
dtype: object

Второй - `iloc` (i - implicit). Он применяется для неявного индекса

In [35]:
df.iloc[1]

'b'

In [36]:
df.iloc[1:3]

3    b
5    c
dtype: object

Вернёмся к вопросу индексирования датафреймов. В примерах будет работать со следующими данными:

In [37]:
df = pd.DataFrame(
    {
        "num_active_versions": pd.Series({ "website": 14, "ios": 4, "android": 8, "windows phone": 1 }), 
        "complexity": pd.Series({ "windows phone": 2.1, "ios": 1.4, "website": 1.0,  "android": 0.9 })
    }
)

df

,num_active_versions,complexity
android,8,0.9
ios,4,1.4
website,14,1.0
windows phone,1,2.1


Давайте рассмотрим, какие возможности при работе с индексами датафреймов существуют:

In [38]:
# выбор столбца по названию
df["num_active_versions"]

android           8
ios               4
website          14
windows phone     1
Name: num_active_versions, dtype: int64

In [39]:
# выбор столбца по полю атрибута
df.complexity

android          0.9
ios              1.4
website          1.0
windows phone    2.1
Name: complexity, dtype: float64

In [40]:
# обратите внимание, что разницы в способах обращения нет
df.complexity is df["complexity"]

True

Обратите внимание, что именно поэтому не рекомендуется именовать столбцы таким образом, чтобы их название совпадало с одним из существующих методов.

Не забывайте, что как и Series, DataFrame можно рассматривать в качестве обобщения словаря, что даёт соответствующие возможности:

In [41]:
# добавить столбец
df["main_lang"] = {
    "android": "kotlin", 
    "ios": "swift", 
    "website": "go", 
    "windows phone": "c#"
}

df

,num_active_versions,complexity,main_lang
android,8,0.9,kotlin
ios,4,1.4,swift
website,14,1.0,go
windows phone,1,2.1,c#


In [42]:
# обращение к значениям датафрейма
df.values

array([[8, 0.9, 'kotlin'],
       [4, 1.4, 'swift'],
       [14, 1.0, 'go'],
       [1, 2.1, 'c#']], dtype=object)

In [43]:
# транспонирование (поменять местами строчки и столбцы)
df.T

,android,ios,website,windows phone
num_active_versions,8,4,14,1
complexity,0.9,1.4,1.0,2.1
main_lang,kotlin,swift,go,c#


Теперь вернёмся к индексерам и их применению к уже датафреймам:

In [45]:
# iloc - implicit indexer (первые 3 строчки и последние два столбца)
df.iloc[:3, -2:]

,complexity,main_lang
android,0.9,kotlin
ios,1.4,swift
website,1.0,go


In [46]:
# loc - explicit indexer (все строчки до website, все столбцы после complexity - ВКЛЮЧАЯ ГРАНИЦЫ)
df.loc[:"website", "complexity":]

,complexity,main_lang
android,0.9,kotlin
ios,1.4,swift
website,1.0,go


In [47]:
# использование масок (выведем все платформы со списком активных версий + основной язык разработки)
df.loc[df.complexity > 1.0, ["num_active_versions", "main_lang"]]

,num_active_versions,main_lang
ios,4,swift
windows phone,1,c#
